In [ ]:
from utils.game_map_class import GameMap
from utils.game_team_class import GameTeam
from utils.strategy_class import Strategy, RandomStrategy, RuleOfThumbStrategy, MonteCarloTreeSearchStrategy, LookaheadRolloutStrategy
from utils.map_setup_functions import setGameBoardRandom, initializeFullRiskMap, setGameBoardRandomWithTroops
from utils.heuristics import BST_Heuristic, EdgeWin
from utils.MCTS import monteCarloTreeSearch, getActions

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
import copy
import math
import random
import numpy as np
from collections import defaultdict

In [ ]:
full_risk_map = initializeFullRiskMap()
team_names = ['red', 'blue']
team_strategies = [RuleOfThumbStrategy, MonteCarloTreeSearchStrategy]
risk_map, teams = setGameBoardRandom(team_names, full_risk_map, team_strategies)

In [ ]:
curr_team = teams[0]
defending_team = teams[1]
counter = 0
while defending_team.hasTeamWon() == False:
    curr_team.playTurn()
    curr_team_temp = curr_team
    curr_team = defending_team
    defending_team = curr_team_temp
    color_map = risk_map.getTeamColorMap()
    nx.draw(risk_map.graph, node_color = color_map, with_labels=True)
    plt.show()
    counter += 1
print("{defender} team has won in {num_turns} turns".format(defender = defending_team.getName(), num_turns = counter // 2))

In [ ]:

winner_array = []
winner_count = []
starter_array = []
#winner_array = np.load('winners.npy').tolist()

#starter_array = np.load('starters.npy').tolist()
for i in range(100):
    full_risk_map = initializeFullRiskMap()
    if i % 2 == 0:
        team_names = ["blue", "red"]
        team_strategies = [MonteCarloTreeSearchStrategy, LookaheadRolloutStrategy]
        print('we start')
        starter_array.append("blue")
    else:
        team_names = ["red", "blue"]
        team_strategies = [LookaheadRolloutStrategy, MonteCarloTreeSearchStrategy]
        print('they start')
        starter_array.append("red")
        
    #risk_map, teams = setGameBoardRandomWithTroops(team_names, full_risk_map, team_strategies)
    risk_map, teams = setGameBoardRandom(team_names, full_risk_map, team_strategies)
    curr_team = teams[0]
    defending_team = teams[1]
    counter = 0
    while defending_team.hasTeamWon() == False:
        curr_team.playTurn()
        curr_team_temp = curr_team
        curr_team = defending_team
        defending_team = curr_team_temp
        color_map = risk_map.getTeamColorMap()
        #nx.draw(risk_map.graph, node_color = color_map, with_labels=True)
        plt.show()
        counter += 1
    
    winner_array.append(defending_team.getName())
    winner_count.append(counter // 2)
    print(i)
    np.save('winners.npy', np.array(winner_array))
    np.save('starters.npy', np.array(starter_array))
    np.save('winners_count.npy', np.array(winner_count))
    print("{defender} team has won in {num_turns} turns".format(defender = defending_team.getName(), num_turns = counter//2))
print(winner_array)
print(winner_count)
print(starter_array)

In [ ]:
i = 0
for winner in winner_array:
    if winner == 'blue':
        i += 1
print(i)

In [ ]:
winners = np.array(winner_array)
starters = np.array(starter_array)
counts = np.array(winner_count)

red = counts[np.where(winners == 'red')[0]]
blue = counts[np.where(winners == 'blue')[0]]
bins = np.linspace(3, 20, 17)

MATLAB_blue = [0, 0.4470, 0.7410]
MATLAB_orange = [0.8500, 0.3250, 0.0980]

plt.hist(red, bins, label='red', facecolor = MATLAB_orange, alpha = .75)
plt.hist(blue, bins, label='blue', facecolor = MATLAB_blue, alpha = .75)

import matplotlib as mpl
mpl.rc('font',family='Times New Roman')

font_size = 12
plt.xlabel('Number of turns until victory', fontsize = font_size)
plt.ylabel('Number of Occurrences', fontsize = font_size)
plt.title('Victories over 100 simulations', fontsize = font_size + 5)
plt.legend({'Rule of Thumb', 'Monte Carlo Tree Search'}, prop={'size': font_size})
plt.savefig("./../images/rule-vs-mcts.png")

In [ ]:
sum = 0
for count in counts[np.where(winners == 'red')[0]]:
    sum += count
total = len(counts[np.where(winners == 'red')[0]])
print(total)
print(sum / total)